# ETL: FIFA World Cup 2022 Dataset

Ce notebook présente les étapes d'un processus ETL (Extract, Transform, Load) sur le jeu de données de la Coupe du Monde 2022.
L'objectif est d'analyser le dataset brut, de nettoyer et de sélectionner les colonnes pertinentes pour une analyse ultérieure.

In [1]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', None)

## 1. Extraction et Chargement des Données
Nous téléchargeons la dernière version du dataset depuis Kaggle via `kagglehub`.

In [2]:
# Download latest version
path = kagglehub.dataset_download("die9origephit/fifa-world-cup-2022-complete-dataset")

print("Path to dataset files:", path)

# Locate and read the CSV
csv_path = None
for root, dirs, files in os.walk(path):
    for file in files:
        if file == "Fifa_world_cup_matches.csv":
            csv_path = os.path.join(root, file)
            break

if csv_path:
    df = pd.read_csv(csv_path)
    print("Dataset loaded successfully.")
else:
    print("File not found.")

Path to dataset files: /home/samchaka/.cache/kagglehub/datasets/die9origephit/fifa-world-cup-2022-complete-dataset/versions/7
Dataset loaded successfully.


### 1.2 Analyse du Dataset (Exploration)
Dans cette étape, nous explorons la structure des données, vérifions les types et recherchons les valeurs manquantes.

In [3]:
# Aperçu des premières lignes
df.head()

,team1,team2,possession team1,possession team2,possession in contest,number of goals team1,number of goals team2,date,hour,category,total attempts team1,total attempts team2,conceded team1,conceded team2,goal inside the penalty area team1,goal inside the penalty area team2,goal outside the penalty area team1,goal outside the penalty area team2,assists team1,assists team2,on target attempts team1,on target attempts team2,off target attempts team1,off target attempts team2,attempts inside the penalty area team1,attempts inside the penalty area team2,attempts outside the penalty area team1,attempts outside the penalty area team2,left channel team1,left channel team2,left inside channel team1,left inside channel team2,central channel team1,central channel team2,right inside channel team1,right inside channel team2,right channel team1,right channel team2,total offers to receive team1,total offers to receive team2,inbehind offers to receive team1,inbehind offers to receive team2,inbetween offers to receive team1,inbetween offers to receive team2,infront offers to receive team1,infront offers to receive team2,receptions between midfield and defensive lines team1,receptions between midfield and defensive lines team2,attempted line breaks team1,attempted line breaks team2,completed line breaksteam1,completed line breaks team2,attempted defensive line breaks team1,attempted defensive line breaks team2,completed defensive line breaksteam1,completed defensive line breaks team2,yellow cards team1,yellow cards team2,red cards team1,red cards team2,fouls against team1,fouls against team2,offsides team1,offsides team2,passes team1,passes team2,passes completed team1,passes completed team2,crosses team1,crosses team2,crosses completed team1,crosses completed team2,switches of play completed team1,switches of play completed team2,corners team1,corners team2,free kicks team1,free kicks team2,penalties scored team1,penalties scored team2,goal preventions team1,goal preventions team2,own goals team1,own goals team2,forced turnovers team1,forced turnovers team2,defensive pressures applied team1,defensive pressures applied team2
0,QATAR,ECUADOR,42%,50%,8%,0,2,20 NOV 2022,17 : 00,Group A,5,6,2,0,0,2,0,0,0,1,0,3,5,3,2,4,3,2,15,8,0,7,3,6,1,4,9,6,520,532,116,127,235,187,169,218,5,8,136,155,86,99,9,13,4,7,4,2,0,0,15,15,3,4,450,480,381,409,9,14,4,4,9,9,1,3,19,17,0,1,6,5,0,0,52,72,256,279
1,ENGLAND,IRAN,72%,19%,9%,6,2,21 NOV 2022,14 : 00,Group B,13,8,2,6,6,2,0,0,6,1,7,3,3,4,10,6,3,2,11,3,5,0,2,3,3,1,11,0,1061,212,207,53,386,86,468,73,16,4,238,101,178,45,25,7,16,4,0,2,0,0,9,14,2,2,809,224,730,154,23,8,7,1,12,3,8,0,16,10,0,1,8,13,0,0,63,72,139,416
2,SENEGAL,NETHERLANDS,44%,45%,11%,0,2,21 NOV 2022,17 : 00,Group A,14,9,2,0,0,2,0,0,0,1,3,3,8,5,7,5,7,4,12,11,4,2,2,2,4,7,13,20,502,506,123,117,230,191,149,198,15,14,151,162,89,96,22,22,15,10,2,1,0,0,13,13,2,1,383,438,313,374,19,25,7,8,9,6,6,7,14,14,0,0,9,15,0,0,63,73,263,251
3,UNITED STATES,WALES,51%,39%,10%,1,1,21 NOV 2022,20 : 00,Group B,6,7,1,1,1,1,0,0,1,0,1,3,4,3,4,5,2,2,14,7,5,2,4,5,4,2,11,7,725,436,149,100,336,172,240,164,12,9,199,174,146,103,23,17,15,8,4,2,0,0,15,10,1,1,569,409,509,321,31,15,4,6,5,8,5,3,11,15,0,1,7,7,0,0,81,72,242,292
4,ARGENTINA,SAUDI ARABIA,64%,24%,12%,1,2,22 NOV 2022,11 : 00,Group C,14,3,2,1,1,2,0,0,0,1,6,2,5,0,10,3,4,0,12,3,4,2,5,3,8,3,18,8,650,268,157,69,177,131,316,68,26,9,191,137,127,68,39,15,25,7,0,6,0,0,7,21,10,1,610,267,529,190,29,9,12,2,5,7,9,2,22,16,1,0,4,14,0,0,65,80,163,361


In [4]:
# Informations sur les colonnes et les types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 88 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   team1                                                  64 non-null     object
 1   team2                                                  64 non-null     object
 2   possession team1                                       64 non-null     object
 3   possession team2                                       64 non-null     object
 4   possession in contest                                  64 non-null     object
 5   number of goals team1                                  64 non-null     int64 
 6   number of goals team2                                  64 non-null     int64 
 7   date                                                   64 non-null     object
 8   hour                                                   64 non-

In [5]:
# Vérification des valeurs nulles
nulls = df.isnull().sum()
print("Colonnes avec des valeurs nulles :")
print(nulls[nulls > 0])

Colonnes avec des valeurs nulles :
Series([], dtype: int64)


In [6]:
# Statistiques descriptives sommaires
df.describe()

,number of goals team1,number of goals team2,total attempts team1,total attempts team2,conceded team1,conceded team2,goal inside the penalty area team1,goal inside the penalty area team2,goal outside the penalty area team1,goal outside the penalty area team2,assists team1,assists team2,on target attempts team1,on target attempts team2,off target attempts team1,off target attempts team2,attempts inside the penalty area team1,attempts inside the penalty area team2,attempts outside the penalty area team1,attempts outside the penalty area team2,left channel team1,left channel team2,left inside channel team1,left inside channel team2,central channel team1,central channel team2,right inside channel team1,right inside channel team2,right channel team1,right channel team2,total offers to receive team1,total offers to receive team2,inbehind offers to receive team1,inbehind offers to receive team2,inbetween offers to receive team1,inbetween offers to receive team2,infront offers to receive team1,infront offers to receive team2,receptions between midfield and defensive lines team1,receptions between midfield and defensive lines team2,attempted line breaks team1,attempted line breaks team2,completed line breaksteam1,completed line breaks team2,attempted defensive line breaks team1,attempted defensive line breaks team2,completed defensive line breaksteam1,completed defensive line breaks team2,yellow cards team1,yellow cards team2,red cards team1,red cards team2,fouls against team1,fouls against team2,offsides team1,offsides team2,passes team1,passes team2,passes completed team1,passes completed team2,crosses team1,crosses team2,crosses completed team1,crosses completed team2,switches of play completed team1,switches of play completed team2,corners team1,corners team2,free kicks team1,free kicks team2,penalties scored team1,penalties scored team2,goal preventions team1,goal preventions team2,own goals team1,own goals team2,forced turnovers team1,forced turnovers team2,defensive pressures applied team1,defensive pressures applied team2
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.0,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.0000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,1.578125,1.109375,11.140625,11.281250,1.109375,1.578125,1.468750,0.984375,0.093750,0.109375,1.171875,0.734375,4.203125,3.750000,4.703125,5.031250,6.93750,6.953125,4.203125,4.328125,13.625000,13.500000,4.921875,4.421875,4.656250,5.156250,4.265625,5.031250,11.656250,12.796875,592.37500,550.218750,126.375000,119.625000,231.500000,212.859375,234.500000,217.734375,11.406250,10.500000,173.46875,166.593750,114.250000,106.484375,18.484375,18.265625,10.156250,9.718750,1.781250,1.750000,0.062500,0.0,12.640625,12.359375,1.968750,1.968750,509.515625,492.109375,437.0000,419.890625,18.093750,18.531250,4.593750,4.078125,6.453125,6.156250,4.484375,4.453125,14.093750,14.390625,0.140625,0.125000,11.593750,11.359375,0.015625,0.015625,71.968750,70.125000,289.750000,293.265625
std,1.551289,1.055856,4.972519,5.807682,1.055856,1.551289,1.563155,0.999876,0.293785,0.314576,1.363407,0.895176,2.527184,2.713868,2.394966,2.911219,3.77912,4.459446,2.470009,2.766321,6.550173,7.287737,2.534240,3.201213,2.852004,3.296071,2.685896,3.141978,5.812463,6.544547,170.21084,169.487694,33.776812,36.660822,70.466698,59.487191,85.887893,101.472843,6.920682,5.614607,32

## 2. Transformation des Données (ETL)
Cette étape consiste à nettoyer les données brutes, à sélectionner les attributs pertinents et à enrichir le dataset si nécessaire.

### 2.1 Sélection des colonnes
Nous conservons uniquement les colonnes pertinentes pour notre analyse (équipes, scores, statistiques clés).

In [7]:
columns_to_keep = [
    'team1', 'team2', 'date', 'hour', 'category',
    'number of goals team1', 'number of goals team2',
    'possession team1', 'possession team2', 'possession in contest',
    'total attempts team1', 'total attempts team2',
    'on target attempts team1', 'on target attempts team2',
    'yellow cards team1', 'yellow cards team2',
    'red cards team1', 'red cards team2',
    'penalties scored team1', 'penalties scored team2'
]

df_selected = df[columns_to_keep].copy()
df_selected.head()

,team1,team2,date,hour,category,number of goals team1,number of goals team2,possession team1,possession team2,possession in contest,total attempts team1,total attempts team2,on target attempts team1,on target attempts team2,yellow cards team1,yellow cards team2,red cards team1,red cards team2,penalties scored team1,penalties scored team2
0,QATAR,ECUADOR,20 NOV 2022,17 : 00,Group A,0,2,42%,50%,8%,5,6,0,3,4,2,0,0,0,1
1,ENGLAND,IRAN,21 NOV 2022,14 : 00,Group B,6,2,72%,19%,9%,13,8,7,3,0,2,0,0,0,1
2,SENEGAL,NETHERLANDS,21 NOV 2022,17 : 00,Group A,0,2,44%,45%,11%,14,9,3,3,2,1,0,0,0,0
3,UNITED STATES,WALES,21 NOV 2022,20 : 00,Group B,1,1,51%,39%,10%,6,7,1,3,4,2,0,0,0,1
4,ARGENTINA,SAUDI ARABIA,22 NOV 2022,11 : 00,Group C,1,2,64%,24%,12%,14,3,6,2,0,6,0,0,1,0


### Justification des choix

Nous choisissons de garder les colonnes suivantes :

1.  **Informations Générales** :
    *   `team1`, `team2` : Les équipes qui s'affrontent.
    *   `date`, `hour`, `category` : Contexte temporel et phase du tournoi (e.g., Groupe, Finale).

2.  **Résultat du Match** :
    *   `number of goals team1`, `number of goals team2` : Indispensable pour connaître le vainqueur.

3.  **Statistiques de Jeu (Performance)** :
    *   `possession team1`, `possession team2` : Indicateur clé de domination.
    *   `total attempts team1`, `total attempts team2` : Volume offensif.
    *   `on target attempts team1`, `on target attempts team2` : Précision et dangerosité réelle.

4.  **Discipline** :
    *   `yellow cards team1`, `yellow cards team2`
    *   `red cards team1`, `red cards team2` : Impact sur le jeu et fair-play.
    *   `penalties scored...` : Pour identifier les matchs aux tirs au but.

Ces colonnes nous permettront de répondre aux questions principales : Qui a gagné ? Qui a dominé ? Le match était-il agressif ?

### 2.2 Nettoyage et Conversion de Types
Les colonnes de possession contiennent le caractère `%` et sont de type objet (string). Nous devons les convertir en valeurs numériques.
De plus, la colonne `date` doit être convertie au format `datetime` pour permettre des analyses temporelles.

In [8]:
import sys
import os
import pandas as pd

# 1. Configuration du chemin pour importer les modules du dossier 'src'
# On récupère le dossier courant et on remonte d'un cran pour trouver 'src'
current_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(current_dir, '..', 'src'))

if src_path not in sys.path:
    sys.path.append(src_path)

# 2. Import de ta fonction externalisée
from cleaning import clean_percentage

# 3. Application du nettoyage sur les colonnes de pourcentage
cols_percent = ['possession team1', 'possession team2', 'possession in contest']

for col in cols_percent:
    if col in df_selected.columns:
        df_selected[col] = df_selected[col].apply(clean_percentage)

# 4. Conversion de la colonne date
df_selected['date'] = pd.to_datetime(df_selected['date'], format='%Y-%m-%d', errors='coerce')

# 5. Affichage des infos pour vérification
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   team1                     64 non-null     object        
 1   team2                     64 non-null     object        
 2   date                      0 non-null      datetime64[ns]
 3   hour                      64 non-null     object        
 4   category                  64 non-null     object        
 5   number of goals team1     64 non-null     int64         
 6   number of goals team2     64 non-null     int64         
 7   possession team1          64 non-null     float64       
 8   possession team2          64 non-null     float64       
 9   possession in contest     64 non-null     float64       
 10  total attempts team1      64 non-null     int64         
 11  total attempts team2      64 non-null     int64         
 12  on target attempts team1

### 2.3 Feature Engineering (Enrichissement)

Création de nouvelles colonnes utiles pour l'analyse :
*   `total_goals` : Nombre total de buts dans le match.
*   `goal_difference` : Écart de buts (valeur absolue).

In [9]:
df_selected['total_goals'] = df_selected['number of goals team1'] + df_selected['number of goals team2']
df_selected['goal_difference'] = abs(df_selected['number of goals team1'] - df_selected['number of goals team2'])

print("Aperçu des données enrichies :")
df_selected.head()

Aperçu des données enrichies :


,team1,team2,date,hour,category,number of goals team1,number of goals team2,possession team1,possession team2,possession in contest,total attempts team1,total attempts team2,on target attempts team1,on target attempts team2,yellow cards team1,yellow cards team2,red cards team1,red cards team2,penalties scored team1,penalties scored team2,total_goals,goal_difference
0,QATAR,ECUADOR,NaT,17 : 00,Group A,0,2,42.0,50.0,8.0,5,6,0,3,4,2,0,0,0,1,2,2
1,ENGLAND,IRAN,NaT,14 : 00,Group B,6,2,72.0,19.0,9.0,13,8,7,3,0,2,0,0,0,1,8,4
2,SENEGAL,NETHERLANDS,NaT,17 : 00,Group A,0,2,44.0,45.0,11.0,14,9,3,3,2,1,0,0,0,0,2,2
3,UNITED STATES,WALES,NaT,20 : 00,Group B,1,1,51.0,39.0,10.0,6,7,1,3,4,2,0,0,0,1,2,0
4,ARGENTINA,SAUDI ARABIA,NaT,11 : 00,Group C,1,2,64.0,24.0,12.0,14,3,6,2,0,6,0,0,1,0,3,1


Les données sont maintenant propres et enrichies, prêtes pour l'analyse.

In [10]:

equipes = pd.concat([df_selected['team1'], df_selected['team2']]).unique().tolist()
print(equipes, len(equipes))

['QATAR', 'ENGLAND', 'SENEGAL', 'UNITED STATES', 'ARGENTINA', 'DENMARK', 'MEXICO', 'FRANCE', 'MOROCCO', 'GERMANY', 'SPAIN', 'BELGIUM', 'SWITZERLAND', 'URUGUAY', 'PORTUGAL', 'BRAZIL', 'WALES', 'NETHERLANDS', 'TUNISIA', 'POLAND', 'JAPAN', 'CROATIA', 'CAMEROON', 'KOREA REPUBLIC', 'ECUADOR', 'IRAN', 'AUSTRALIA', 'SAUDI ARABIA', 'CANADA', 'COSTA RICA', 'GHANA', 'SERBIA'] 32


### 2.4 Adaptation au Schéma SQL (Matches)

Nous transformons le DataFrame pour correspondre exactement à la structure de la table SQL `matches` cible. 
Cela inclut la génération d'IDs pour les équipes, le renommage des colonnes et la création des indicateurs manquants.

In [11]:
# Génération d'IDs pour les équipes (mapping simple basé sur l'ordre alphabétique)
# Dans un cas réel, ces IDs viendraient d'une table 'teams' déjà peuplée
unique_teams = sorted(pd.concat([df_selected['team1'], df_selected['team2']]).unique())
# team_mapping = {team: i+1 for i, team in enumerate(unique_teams)}

df_matches = df_selected.copy()

# Création des Foreign Keys
df_matches['home_team_id'] = df_matches['team1']
df_matches['away_team_id'] = df_matches['team2']

# Renommage des colonnes
df_matches = df_matches.rename(columns={
    'number of goals team1': 'home_result',
    'number of goals team2': 'away_result',
    'category': 'round'
})

# Calcul de la colonne 'result'
conditions = [
    df_matches['home_result'] > df_matches['away_result'],
    df_matches['home_result'] < df_matches['away_result']
]
choices = ['home_team', 'away_team']
df_matches['result'] = np.select(conditions, choices, default='draw')

# Gestion des booléens
df_matches['penalties'] = (df_matches['penalties scored team1'] + df_matches['penalties scored team2']) > 0
df_matches['extra_time'] = df_matches['penalties'] # Assumption simplifiée
df_matches['replay'] = False

# Colonnes statiques ou manquantes
df_matches['edition'] = 2022
df_matches['city'] = None # Information non disponible dans ce dataset
df_matches['id_stadium'] = None # Information non disponible

# Génération d'un ID de match séquentiel
df_matches['id_match'] = range(1, len(df_matches) + 1)

# Sélection et ordonnancement final des colonnes selon le schéma SQL
final_cols = [
    'id_match', 
    'home_team_id', 'away_team_id', 
    'home_result', 'away_result', 'result', 
    'extra_time', 'penalties', 'replay', 
    'date', 'round', 'city', 'id_stadium', 'edition'
]

df_matches_final = df_matches[final_cols].copy()

print("Aperçu du DataFrame final prêt pour l'insertion SQL :")
df_matches_final.head()

Aperçu du DataFrame final prêt pour l'insertion SQL :


,id_match,home_team_id,away_team_id,home_result,away_result,result,extra_time,penalties,replay,date,round,city,id_stadium,edition
0,1,QATAR,ECUADOR,0,2,away_team,True,True,False,NaT,Group A,None,None,2022
1,2,ENGLAND,IRAN,6,2,home_team,True,True,False,NaT,Group B,None,None,2022
2,3,SENEGAL,NETHERLANDS,0,2,away_team,False,False,False,NaT,Group A,None,None,2022
3,4,UNITED STATES,WALES,1,1,draw,True,True,False,NaT,Group B,None,None,2022
4,5,ARGENTINA,SAUDI ARABIA,1,2,away_team,True,True,False,NaT,Group C,None,None,2022


In [12]:
import requests
import json
import os  # Nécessaire pour gérer les chemins de dossiers

# 1. URL pour les données en Anglais
url = "https://api.fifa.com/api/v3/calendar/matches?language=en&count=500&idSeason=255711"

# 2. Configuration (Headers)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Origin': 'https://www.fifa.com',
    'Referer': 'https://www.fifa.com/'
}

print("⏳ Téléchargement des données en Anglais...")

try:
    # 3. Récupération
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    # --- PARTIE MODIFIÉE : GESTION DU DOSSIER DATA ---
    
    # On définit le nom du fichier
    nom_fichier = "matches_wc2022_en.json"
    
    # On définit le chemin : ".." signifie "remonter d'un dossier", puis aller dans "data"
    # os.path.join permet que ça marche sur Windows (anti-slash) et Mac/Linux (slash)
    chemin_dossier = os.path.join("..", "data/raw")
    chemin_complet = os.path.join(chemin_dossier, nom_fichier)

    # Sécurité : On crée le dossier s'il n'existe pas (même si vous l'avez déjà)
    os.makedirs(chemin_dossier, exist_ok=True)
    
    # 4. Sauvegarde dans le dossier spécifique
    with open(chemin_complet, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
    # On affiche le chemin absolu pour confirmer l'emplacement
    print(f"✅ Succès ! Fichier enregistré ici :")
    print(os.path.abspath(chemin_complet))

except Exception as e:
    print(f"❌ Erreur : {e}")

⏳ Téléchargement des données en Anglais...
✅ Succès ! Fichier enregistré ici :
/home/samchaka/simplon/projet_2_short_kings/Brief-2-ETL-donnees-footballistiques-Short-Kings/data/raw/matches_wc2022_en.json


In [13]:
import json
import os
import pandas as pd
import numpy as np
import sys

# 1. Configuration du chemin src (si pas déjà fait dans une cellule précédente)
current_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# IMPORT DE LA NOUVELLE FONCTION
from cleaning import clean_round_name

# 2. Chargement du JSON
json_path = os.path.join('..', 'data/raw', 'matches_wc2022_en.json')
with open(json_path, 'r', encoding='utf-8') as f:
    wc_data = json.load(f)

# 3. Extraction
extracted_data = []

for match in wc_data['Results']:
    # Noms d'équipes en Title Case (pas UPPERCASE)
    home = match['Home']['TeamName'][0]['Description'].title()
    away = match['Away']['TeamName'][0]['Description'].title()
    
    stage = match['StageName'][0]['Description']
    group = match['GroupName'][0]['Description'] if match['GroupName'] else None
    
    # Logique pour déterminer le nom brut avant nettoyage
    round_raw = group if (stage == "First stage" and group) else stage
    
    # Récupération des scores
    score_board_home = match.get('HomeTeamScore', 0)
    score_board_away = match.get('AwayTeamScore', 0)
    pen_home = match.get('HomeTeamPenaltyScore', 0)
    pen_away = match.get('AwayTeamPenaltyScore', 0)
    
    is_penalties = (pen_home > 0 or pen_away > 0)
    total_calc_home = score_board_home + pen_home
    total_calc_away = score_board_away + pen_away
    
    # Extraction de la date depuis le JSON
    match_date = match.get('Date')  # Format: "2022-11-20T16:00:00Z"

    extracted_data.append({
        'home_team_id': home,
        'away_team_id': away,
        'date': match_date,  # Date extraite du JSON
        'round': clean_round_name(round_raw),
        'city': match['Stadium']['CityName'][0]['Description'],
        'id_stadium': match['Stadium']['Name'][0]['Description'],
        'extra_time': is_penalties,
        'penalties': is_penalties,
        'home_result': score_board_home, 
        'away_result': score_board_away,
        'temp_total_home': total_calc_home,
        'temp_total_away': total_calc_away
    })

# 4. Création du DataFrame depuis le JSON
df_json = pd.DataFrame(extracted_data)

# Conversion de la date en format YYYY-MM-DD
df_json['date'] = pd.to_datetime(df_json['date'], utc=True).dt.strftime('%Y-%m-%d')

# 5. Calcul du résultat (avec TAB inclus)
conditions = [
    (df_json['temp_total_home'] > df_json['temp_total_away']),
    (df_json['temp_total_home'] < df_json['temp_total_away'])
]
choices = ['home_team', 'away_team']
df_json['result'] = np.select(conditions, choices, default='draw')

# 6. Création du DataFrame final avec le schéma cible (14 colonnes)
df_matches_final = pd.DataFrame({
    'id_match': range(1, len(df_json) + 1),
    'home_team_id': df_json['home_team_id'],
    'away_team_id': df_json['away_team_id'],
    'home_result': df_json['home_result'],
    'away_result': df_json['away_result'],
    'result': df_json['result'],
    'extra_time': df_json['extra_time'],
    'penalties': df_json['penalties'],
    'replay': False,  # Pas de match rejoué en 2022
    'date': df_json['date'],
    'round': df_json['round'],
    'city': df_json['city'],
    'id_stadium': df_json['id_stadium'],
    'edition': 2022
})

# Affichage de vérification
print(f"Shape: {df_matches_final.shape} (attendu: 64 x 14)")
print(f"Colonnes: {list(df_matches_final.columns)}")
print(f"\nDates (aperçu):")
print(df_matches_final[['home_team_id', 'away_team_id', 'date', 'round']].head())
print(f"\nMatchs avec TAB:")
display(df_matches_final[df_matches_final['penalties'] == True][['home_team_id', 'away_team_id', 'home_result', 'away_result', 'result', 'date', 'round']])

Shape: (64, 14) (attendu: 64 x 14)
Colonnes: ['id_match', 'home_team_id', 'away_team_id', 'home_result', 'away_result', 'result', 'extra_time', 'penalties', 'replay', 'date', 'round', 'city', 'id_stadium', 'edition']

Dates (aperçu):
    home_team_id  away_team_id        date        round
0          Qatar       Ecuador  2022-11-20  Group Stage
1        England          Iran  2022-11-21  Group Stage
2        Senegal   Netherlands  2022-11-21  Group Stage
3  United States         Wales  2022-11-21  Group Stage
4      Argentina  Saudi Arabia  2022-11-22  Group Stage

Matchs avec TAB:


,home_team_id,away_team_id,home_result,away_result,result,date,round
52,Japan,Croatia,1,1,away_team,2022-12-05,Round of 16
54,Morocco,Spain,0,0,home_team,2022-12-06,Round of 16
56,Croatia,Brazil,1,1,home_team,2022-12-09,Quarter-finals
57,Netherlands,Argentina,2,2,away_team,2022-12-09,Quarter-finals
63,Argentina,France,3,3,home_team,2022-12-18,Final


In [14]:
# Export du DataFrame final
from pathlib import Path

# Création du dossier processed si nécessaire
Path("../data/processed").mkdir(parents=True, exist_ok=True)

# Export
df_matches_final.to_csv('../data/processed/df_matches_final.csv', index=False)

# Vérifications finales
print(f"✅ Export terminé : df_matches_final.csv")
print(f"   - {len(df_matches_final)} matchs exportés")
print(f"   - {len(df_matches_final.columns)} colonnes : {list(df_matches_final.columns)}")
print(f"   - Dates : {df_matches_final['date'].notna().sum()} / {len(df_matches_final)} remplies")
print(f"   - Matchs avec prolongation : {df_matches_final['extra_time'].sum()}")
print(f"   - Matchs avec TAB : {df_matches_final['penalties'].sum()}")

✅ Export terminé : df_matches_final.csv
   - 64 matchs exportés
   - 14 colonnes : ['id_match', 'home_team_id', 'away_team_id', 'home_result', 'away_result', 'result', 'extra_time', 'penalties', 'replay', 'date', 'round', 'city', 'id_stadium', 'edition']
   - Dates : 64 / 64 remplies
   - Matchs avec prolongation : 5
   - Matchs avec TAB : 5
